In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [2]:
pic_one = r"C:\Users\jiyun\Desktop\Jiyu\2020-2021\Winter\CSC413 - Neural Networks and Deep Learnin\Final Project\Image-Captioning-Reproduction\data\val2014\val2014\COCO_val2014_000000000285.jpg"
pic_two = r"C:\Users\jiyun\Desktop\Jiyu\2020-2021\Winter\CSC413 - Neural Networks and Deep Learnin\Final Project\Image-Captioning-Reproduction\data\val2014\val2014\COCO_val2014_000000000502.jpg"
# pic_one = str(path_one)
# pic_two = str(path_two)

In [3]:
# Load the pretrained model
model = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

In [4]:
# Set model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

C:\Users\jiyun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torchvision\transforms\transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [6]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
#     import pdb; pdb.set_trace()
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [7]:
pic_one_vector = get_vector(pic_one)
pic_two_vector = get_vector(pic_two)

In [9]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(pic_one_vector.unsqueeze(0),
              pic_two_vector.unsqueeze(0))
cos_sim

tensor([0.7028])

In [10]:
pic_one_vector

tensor([1.7826e+00, 2.5210e-01, 1.2422e+00, 4.7411e-01, 1.6703e+00, 1.7719e-01,
        1.2750e+00, 2.0410e+00, 6.7764e-01, 8.9163e-01, 4.4451e-01, 9.1998e-01,
        3.4095e+00, 7.9513e-02, 1.5283e+00, 2.8774e-01, 7.4838e-01, 1.9869e+00,
        7.4925e-01, 6.1960e-02, 2.0624e-01, 1.8651e+00, 1.8422e+00, 1.2656e+00,
        4.2586e-01, 0.0000e+00, 8.6108e-01, 1.3742e+00, 4.9021e-01, 2.4977e+00,
        1.0179e-01, 2.6996e-03, 2.0250e+00, 3.1927e-01, 5.2740e-01, 8.6557e-01,
        2.2881e-01, 1.7459e+00, 3.6572e-01, 2.0296e+00, 3.9448e-02, 1.6928e+00,
        1.4204e+00, 8.0132e-01, 2.6842e-01, 1.1884e+00, 3.3753e+00, 2.2603e-01,
        2.1726e-01, 7.1075e-01, 1.5467e+00, 1.1236e+00, 3.6591e-01, 9.7498e-01,
        5.4708e-01, 3.7703e-01, 3.4217e+00, 9.9709e-02, 3.9140e-01, 2.0770e+00,
        1.4149e+00, 1.2824e+00, 1.9456e+00, 2.5578e+00, 4.9412e-02, 1.8026e+00,
        2.8715e-01, 1.1501e-01, 5.6986e-01, 2.5310e-01, 1.7087e+00, 3.6262e-02,
        3.1162e-01, 2.0679e+00, 1.0460e+

***

In [60]:
from torchvision.datasets import CocoCaptions
from pycocotools.coco import COCO
from tqdm import tqdm
import random

In [31]:
coco_dataset = CocoCaptions(
    root="../data/val2014/val2014", 
    annFile="../annotations/annotations_trainval2014/annotations/captions_val2014.json"
)

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!


In [63]:
random.choice(coco_dataset)

(<PIL.Image.Image image mode=RGB size=480x640 at 0x7F4553722460>,
 ['a green and blue parking meter siting on the sidewalk',
  'A green parking meter sits on the side of the road',
  'A green parking meter on a city street.',
  'A parking meter sitting idly on a sidewalk.',
  'Multi colored parking meter with a parking sign on it.'])

In [39]:
coco_dataset.ids

[42,
 73,
 74,
 133,
 136,
 139,
 143,
 164,
 192,
 196,
 208,
 241,
 257,
 283,
 285,
 294,
 328,
 338,
 357,
 359,
 360,
 387,
 395,
 397,
 400,
 415,
 428,
 459,
 472,
 474,
 486,
 488,
 502,
 520,
 536,
 544,
 564,
 569,
 589,
 590,
 599,
 623,
 626,
 632,
 636,
 641,
 661,
 675,
 692,
 693,
 699,
 711,
 715,
 724,
 730,
 757,
 761,
 764,
 772,
 775,
 776,
 785,
 802,
 810,
 827,
 831,
 836,
 872,
 873,
 885,
 923,
 939,
 962,
 969,
 974,
 985,
 987,
 999,
 1000,
 1029,
 1063,
 1064,
 1083,
 1089,
 1103,
 1138,
 1146,
 1149,
 1153,
 1164,
 1171,
 1176,
 1180,
 1205,
 1228,
 1244,
 1268,
 1270,
 1290,
 1292,
 1296,
 1340,
 1342,
 1347,
 1353,
 1369,
 1374,
 1393,
 1398,
 1404,
 1425,
 1436,
 1442,
 1448,
 1464,
 1490,
 1503,
 1525,
 1532,
 1554,
 1561,
 1563,
 1573,
 1577,
 1584,
 1590,
 1591,
 1592,
 1599,
 1626,
 1647,
 1655,
 1667,
 1668,
 1675,
 1682,
 1700,
 1722,
 1739,
 1757,
 1761,
 1779,
 1799,
 1803,
 1818,
 1840,
 1841,
 1852,
 1856,
 1869,
 1903,
 1948,
 1955,
 1960,
 19

In [55]:
count = 0
for ids, x in tqdm(zip(coco_dataset.ids, coco_dataset)):
    count +=1
    print(ids)
    break

0it [00:00, ?it/s]

42


In [29]:
len(coco_dataset)

40504

In [ ]:
count